# Нормализация сцен Ландсат

Даны разновременные сцены Ландсат, покрывающие одну и ту же территорию. Яркости пикселей разных сцен изначально несравнимы между собой вследствие различий в условиях съемки. Требуется промасштабировать яркости пикселей этих сцен таким образом, чтобы появилась возможность сравнивать их между собой и отслеживать динамику изменений яркостей. Эта процедура называется относительной нормализацией.

Нормализация, применяемая в данном документе, в основном следует методике из статьи Du Y., Teillet P. M., Cihlar J. Radiometric normalization of multitemporal high-resolution satellite images with quality control for land cover change detection //Remote sensing of Environment. – 2002. – Т. 82. – №. 1. – С. 123-134.

Основные положения статьи:

1. Если выполняются определенные условия, в частности, если нелинейные эффекты от снимка к снимку незначительны по сравнению с линейными, и искажения равномерно распределены в пространстве, то можно производить линейную нормализацию. (Более точная формулировка условия дана в следующем разделе.)
2. Для линейной нормализации следует выбрать псевдоинвариантные объекты (PIF) и линейно преобразовать яркости пикселей таким образом, чтобы статистики яркостей (среднее и дисперсия), вычисленные по выбранным PIF совпадали у различных сцен.
3. PIF можно выбирать вручную, но это долго и не объективно. В статье предлагается процедура автоматического выбора PIF. Поскольку эта часть статьи наиболее интересна для нас и на ней основана вся методика, то она рассматривается ниже в отдельном разделе.

Тем не менее, несмотря на то, что в основе лежит данная статья, нами были введены некоторые модификации в процедуру нормализации.

Документ состоит из трех основных частей (не считая приложений):
 * [Теоретическая часть](#Theory) содержит краткое изложение статьи, лежащей в основе.
 * [Практическая часть](#Practic) содержит исследование предолженного в статье алгоритма и его устойчивость.
 * [Процедура нормализации](#Implementation) содержит выводы по процедуре нормализации и окончательную реализацию этой процедуры.


<a id='Theory'></a>
## Методика выбора PIF: теория

### Ограничения метода

Предположим, что выполняются следующие условия:

1. Отражающая способность объектов, выбранных в качестве PIF и используемых в процедуре нормализации, является постоянной в течении анализируемого периода времени.
2. Все изменения яркостей пикселей (DN), которые фиксируются на снимках для выбранных PIF, не являются реальным изменением объектов на местности.
3. Все изменения DN в течении анализируемого периода времени линейны и пространственно однородны для всех PIF, учавствующих в процедуре нормализации.
4. Все эффекты, приводящие к изменениям DN, независимы друг от друга и их результирующая величина представляет собой линейную комбинацию отдельных эффектов.
5. Реальные изменения объектов сцены не являются массовыми.

В этом случае процедура относительной нормализации сцен может быть реализована на базе линейного преобразования. 

**Замечание.** В рассматриваемой статье предполагается, что это DN (при отосительной нормализации не обязательно переходить от DN к ToAR, поскольку этот переход линеен и, следовательно, будет учтен при нормализации), но мы будем работать не с DN, а с ToAR и данными, прошедшими топографическую коррекцию. Этот выбоор обусловлен тем, что нам необходимо произвести маскирование облачности (оно оперирует ToAR), а также тем, что в случае горных условий Дальнего Востока очень желательно произвести топографическую коррекцию, поскольку нужно нивелировать влияние склонов и положения Солнца (без этого возникнет явное противоречие с условием пространственной однородности).

### Нормализация

Пусть Q(i) -- яркость пикселя PIF номер i. Линейная нормализация подразумевает преобразование изображений 1, 2, ..., j, ..., m по следующей формуле:
$$
N_j(i) = Q_j(i)\times \alpha_i + \beta_i = Q_j(i)\times \frac{\sigma_{ref}^{(Q)}}{\sigma_j^{(Q)}} +  \overline{Q}_{ref} \frac{\sigma_{ref}^{(Q)}}{\sigma_j^{(Q)}}\times \overline{Q}_j
$$
где $N_j(i)$ - нормированное значение i-го PIF в j-м изображении, $\overline{Q}_{ref}$, $\sigma_{ref}^{(Q)}$ - желаемые статистики средних яркостей и дисперсии, рассчитанных по всем PIF (они могут быть произвольными константами или же вычислены на базе произвольного изображения), а $\overline{Q}_j$ и $\sigma_j^{(Q)}$ - средняя яркость и дисперсия для PIF в изображении j.

### Стратегии выбора PIF

На практике PIF выбираются или руками (что нам не интересно), или автоматически. Будем рассматривать второй способ. 

Выше были сформированы условия линейности, при выполнении которых мы можем расчитывать на то, что PIFы между собой связаны линейным законом. Это означает, что если мы возьмем два изображения и преобразуем их при помощи метода главных компонент, то основная информация будет сконцентрирована вдоль одной оси (первой главной компоненты), вдоль второй оси будут собраны шумы и изменения. Это обосновывает (в статье рассказывается несколько подробнее) следующую процедуру поиска PIF:

1. Очистка данных от заведомо нестабильных пикселей: (а) применить порог для удаления облачных пикселей; (б) удалить пиксели, соответствующие водным объектам (они быстро меняются); (в) удалить все пиксели, для которых разность между первым и вторым изображением выше некоторого порога (они сигнализируют о потенциальном изменении, опасно выбирать их в качестве PIF).
2. Произвести преобразование оставшихся пикселей выбранной пары изображений методом главных компонент.
3. Все пиксели, лежащие вдоль главной оси на расстоянии не большем $l$ единиц (т.е. $-l < minor < l$, где minor - вторая главная компонента) маркируются как потенциальные PIF.
4. Если выбранные PIF удовлетворяют критериям качества (см. ниже), то процедура поиска останавливается. Если нет, то уменьшается величина $l$ (а также, возможно, ужесточаются пороговые значения из п.1) и производится повторный поиск PIF при новых параметрах.
5. Вычисляются среднее и дисперсия яркостей выбранных PIF (величины $\overline{Q}_j$ и $\sigma_j^{(Q)}$), которые в дальнейшем используются при нормализации.

**Замечание.** В статье рассматриваются, по всей видимости, только одноканальные изображения или же многоканальные, но при этом каждый канал анализируется отдельно от остальных. В случае многоканальных изображений, вероятно, такой подход приведет к некторому искажению отношения каналов: $B_k/B_m \approx N_k/N_m (?)$ (Другими словами, после нормировки спектральные вектора подвергнутся некоторому повороту; небольшой поворот не страшен, он может объясняться шумами, но нет гарантии, что этот поворот не окажется больше, чем следует). Т.е. по-хорошему, нужно доработать процедуру, предоженную в статье, для того, чтобы можно было использовать многоканальные изображения без искажений спектральных векторов объектов. (Ну или убедиться, что данная процедура не приводит к такому искажению). На текущем этапе оставим все как есть.

### Оценка качества PIF

Для оценки качества PIF авторы предлагают пройти несколько стадий/способов оценки:

1. Посчитать коэффициент корреляции между выбранными PIF. Авторы предлагают считать набор не подходящим, если коэффициент не превышает значение 0.9. Этот метод должен неплохо срабатывать на этапе выбора порогов разности изображений и/или порогов облачности и водных объектов.
2. После применения метода главных компонент оценкой качества выбранных PIF может служить величина наклона главной оси: чем ближе она к 45 градусам, тем лучше. Итеративным подбором величины $l$ можно добиться постепенного приближения наклона главной оси к желаемому.
3. Третий способ применим, если производится нормализация не одной пары, а нескольких пар снимков. Например, имеются три снимка A, B и C. Тогда можно применить метод главных компонент к парам AB, AC, BC. При этом не исключено, что параметры нормализации, полученные для разных пар могут быть разными (зная парамаметры нормализации для пар AB и BC можно получить параметры для AC, которые могут противоречить вычисленным напрямую для AC значениям). Поэтому еще одним критерием качества может служить близость параметров нормализации, вычисленных разными способами.

<a id='Practic'></a>
## Нормализация: практика

### Процедура выбора PIF

Создадим функцию, которая будет брать на вход названия двух растров и искать на них PIF, результирующие PIF будут сохранены в виде растра с заданным именем. Функция вернет словарь с описанием процесса работы:
 
 * b: коффициент регрессии между PIF (тангенс угла наклона линии регрессии), чем ближе к единице, тем лучше;
 * N: число обнаруженных PIF;
 * iterations: число выполненных итераций;
 * good_quality: True, если процедура сошлась, и False в противном случае.

**TODO:** Сейчас реализация просто даляет маску (растр MASK), если она была. По-хорошему, нужно сделать проверку того, что растр маски присутствует и восстанавливать его после работы функции.

Функция управляется несколькими параметрами. Наиболее важные из них:

* regression_eps описывает допустимое отклонение регрессионной кривой от 45 градусов наклона; если по завершении работы алгоритма тангенс угла наклона отличается от тагнеса 45 градусов более, чем на эту величину (по модулю), считаем, что алгритм не сошелся и PIF не достоверны.
* min_pixels=1000 число пикселей (PIF), при которых результат еще считается достоверным; с каждой итерацией число потенциальных PIF уменьшается из-за более жестких накладываемых ограничений; по достижении count(PIF) < min_pixels работа алгоритма прерывается, считаем, что алгоритм не сошелся;
* max_iteration максимально допустимое число итераций, при превышении этого параметра, считаем, что алгоритм не сошелся. 

In [ ]:
def find_pif(grass, rast1, rast2, result, 
             diff_eps=0.2, diff_eps_rate=0.9,
             l_eps=0.01, l_eps_rate=0.9,
             regression_eps=0.01,
             min_pixels=1000,
             max_iteration=25):
    
    ###### Params for flow control and quality assesment #########
    # diff_eps          Init threshold for the difference between input rasters
    # diff_eps_rate     Rate for decrease diff_eps param during every iteration.
    
    # l_eps             Init threshold for l (l < minor < l)
    # l_eps_rate        Rate for decrease l_eps param during every iteration.
    
    # Tolerance for regression line coef: abs(b - 1) < regression_eps
    # where PIF_rast1 = a + b*PIF_rast2; b(very_good_quality) == 1
    # regression_eps  
    
    # If count (unmasked pixels) < then min_pixels, then exit. (good_quality := False)
    # min_pixels    
    
    # max_iteration     Maximum count of iterations. 
    
    ###### End of params for flow control and quality assesment #########
    
    
    # Check if MASK raster exists
    mask_exists = True
    try:
        _ = grass.raster_info('MASK')
    except:
        mask_exists = False
    
    # Save MASK map (if present) to temp raster
    if mask_exists:
        tmp_MASK_name = 'MASK' + uuid.uuid4().hex
        grass.run_command('g.copy', rast="MASK,%s" % (tmp_MASK_name, ))
    
    good_quality = False  # Are the PIFs good?
    
    current_iteration = 0
    while (not good_quality) and (current_iteration < max_iteration):
        current_iteration += 1
        

        # Step 1
        try:
            tmp_diff_name = 'tmp' + uuid.uuid4().hex
            expr = '%s = abs(%s - %s)' % (tmp_diff_name, rast1, rast2)
            grass.run_command('r.mapcalc', expression=expr)
            grass.run_command('r.mapcalc', 
                              expression="MASK=if(%s < %s, 1, null())" % (tmp_diff_name, diff_eps),
                              overwrite=True
            )
        finally:
            grass.run_command('g.remove', type='rast', name=tmp_diff_name, flags='f')
                
        try:
            # Step 2
            tmp_pca_name = 'tmp' + uuid.uuid4().hex
            grass.run_command('i.pca',
                              input=','.join([rast1, rast2]),
                              output=tmp_pca_name,
                              rescale='0,0'
                             )
            # Step 3
            # PCA minor is stored in 'xxxxx.2' raster (we have 2 rasters only)
            expr = '%s = if(abs(%s) > %s, null(), 1)' % ('MASK', tmp_pca_name+'.2', l_eps)
            grass.run_command('r.mapcalc', expression=expr, overwrite=True)
        finally:
            grass.run_command('g.remove', type='rast', pattern=tmp_pca_name+'*', flags='f')
    
        # Step 4
        # regression line quality:
        coefs = grass.parse_command('r.regression.line',
                          mapx=rast1, mapy=rast2, flags='g'
                        )

        N = int(coefs['N'])
        b = float(coefs['b'])

        if abs(b - 1) < regression_eps:
            # PIF are found
            good_quality = True
            grass.run_command('g.copy', rast=','.join(['MASK', result]), overwrite=True)
            
        # Restore initial MASK (if present) or remove temp MASK
        if mask_exists:
            grass.run_command('g.copy', rast="%s,MASK" % (tmp_MASK_name, ), overwrite=True)
        else:
            grass.run_command('r.mask', flags='r')
        
        
        if N < min_pixels:
            # Next iterations don't increase count of umnasked pixels
            good_quality = False
            break
        
        
        # Update thresholds
        diff_eps *= diff_eps_rate
        l_eps *= l_eps_rate
    
    result_info = dict(
        good_quality=good_quality,
        N=N,
        b=b,
        iterations=current_iteration
    )
                              
    return result_info


In [ ]:
import os
import uuid

import numpy as np

from tqdm import tqdm_notebook

import utilites
reload(utilites)

from utilites import (
    get_grassdata_path,
    get_location_name,
    get_ll_location_name,
    get_location_path,
)


from grasslib import GRASS

grs = GRASS(gisbase='/usr/lib/grass70', 
            dbase=get_grassdata_path(), 
            location=get_location_name()
)

grs.grass.run_command('g.mapset', mapset='landsat', flags='c')
print grs.grass.read_command('g.mapset', flags='p')

Пример запуска:

In [ ]:
rast1 = 'clean.topo.toar_LC81120272014106LGN00_B2'
rast2 = 'clean.topo.toar_LC81120272014090LGN00_B2'
grs.grass.run_command('g.region', rast=rast1, res='120')

grs.grass.run_command('r.mask', flags='r')
grs.grass.run_command('r.mapcalc', expression="MASK=forest.mask.30@treecover", overwrite=True)

find_pif(grs.grass, rast1, rast2, 'tmp_PIF')

Напишем функцию, которая будет брать растр, PIF, желаемые величины среднего и дисперсии и возвращать коэффициенты нормализации intercept и factor ($N_{norm} = factor \times N + intercept$):

In [ ]:
def get_norm_coef(grass, rast, pif, mean_ref=0.0, var_ref=1.0):
    # Check if MASK raster exists
    mask_exists = True
    try:
        _ = grass.raster_info('MASK')
    except:
        mask_exists = False
    
    # Save MASK map (if present) to temp raster
    if mask_exists:
        tmp_MASK_name = 'MASK' + uuid.uuid4().hex
        grass.run_command('g.rename', rast="MASK,%s" % (tmp_MASK_name, ))
        
        
    try:
        grass.run_command('r.mask', raster=pif)
        stat = grass.parse_command('r.univar', map=rast, flags='g')
    finally:
        grass.run_command('r.mask', flags='r')
    
    if mask_exists:
        grass.run_command('g.rename', rast="%s,MASK" % (tmp_MASK_name, ))
        
    var = float(stat['variance'])
    mean = float(stat['mean'])
    
    factor = var_ref/var
    res = dict(
        intercept=(mean_ref - mean) * factor,
        factor=factor
    )
    
    return res

In [ ]:
grs.grass.run_command('r.mask', raster='forest.mask.30@treecover', overwrite=True)
print get_norm_coef(grs.grass, rast1, 'tmp_PIF')
print get_norm_coef(grs.grass, rast2, 'tmp_PIF')

### Подбор параметров нормализации

В алгоритме поиска PIF используются несколько параметров. Наибольшее влияние на результат должны оказывать, в первую очередь, параметры, определяющие критерии остановки работы алгоритма: regression_eps, min_pixels, max_iteration.

В этом разделе изучается достоверность нормирующих коэффициентов и влияние параметров алгоритма на коэффициенты нормализации. В качестве примера возмем одну сцену (path/row) и один канал.

Начнем с того, что вычислим нормирующие коэффициенты для одного канала одной сцены всеми доступными альтернативными способами. Затем постараемся понять, насколько сильно отличаются полученные коэффициенты и почему.

In [ ]:
prefix = 'clean.topo.toar_LC8112027'
rasters = grs.grass.list_strings(type='rast', pattern=prefix+'*_B1')

In [ ]:
grs.grass.run_command('r.mask', flags='r')
grs.grass.run_command('r.mapcalc', expression="MASK=forest.mask.30@treecover", overwrite=True)

norm_params = dict()
iter_count = len(rasters) * (len(rasters) -1 ) / 2

with tqdm_notebook(total=iter_count) as pbar:
    for i in range(len(rasters)):
        for j in range(len(rasters)):
            if i > j:
                rast1, rast2 = rasters[i], rasters[j]
                print rast1, rast2
                params = find_pif(grs.grass, rast1, rast2, 'tmp_PIF', regression_eps=0.01)
                print params
                if params['good_quality']:
                    try:
                        norm_params[rast1][rast2] = get_norm_coef(grs.grass, rast1, 'tmp_PIF')
                    except:
                        norm_params[rast1] = {rast2: get_norm_coef(grs.grass, rast1, 'tmp_PIF')}

                    try:
                        norm_params[rast2][rast1] = get_norm_coef(grs.grass, rast2, 'tmp_PIF')
                    except:
                        norm_params[rast2] = {rast1: get_norm_coef(grs.grass, rast2, 'tmp_PIF')}
                        
                pbar.update(1)
pbar.close()

Результат выполнения команды очень многословен, см. его [в приложении 1](#appendix1). Для нас там важно то, что:
 
 * часть коэффициентов не была найдена из-за недостаточного количества итераций, соответственно, при большем числе допустимых итераций или меньшей требуемой точности эти коэффициенты можно было бы определить;
 * часть коээфициентов не была найдена из-за недостаточного количества PIF. 

Посмотрим на полученные нормировочные коэффициенты. "Сырые" данные лежат в [в приложении 2](#appendix2), для удобства анализа сведем их в таблицу.

В строках таблицы будут перечислены нормируемые растры, в заголовках столбцов - растры, которые использовались для поиска PIF совместно с растром, указанном в строке. В ячейке лежат полученные коэффициенты нормирования, вычисленные по PIF для указанной пары растров. Если для данной пары растров алгоритм поиска PIF не сошелся, в ячейке стоит прочерк.

Ожидается, что коэффициенты нормирования растра должны быть приблизительно одинаковыми не зависимо от второго растра использованного при поиске PIF. Т.е. в каждой строке должны быть приблизительно одинаковые значения.

#### Параметры нормализации на всех данных (без маскирования)

|LC81120272014074|LC81120272014090|LC81120272014106|LC81120272014346|LC81120272014362|LC81120272015045|LC81120272015077|LC81120272015093|LC81120272015109|LC81120272015157|LC81120272015189|LC81120272015333|LC81120272015349|LC81120272015365|LC81120272016032|LC81120272016048|LC81120272016064|LC81120272016080|LC81120272016096
-|--|--|--|--|--|--|--|--|--|--|--|--|--|--|--|--|--|--|-
LC81120272014074|-|f=163.867578279; i=-32.4920630756|-|f=49.5817879123; i=-21.1954656682|f=62.4287105015; i=-26.6610331836|f=29.5464527696; i=-10.3111348373|f=38.5220610646; i=-17.4218223919|-|f=61.6918746854; i=-16.0135007081|f=1019.43713808; i=-165.86428063|f=3476.95458396; i=-518.415259479|f=66.8879278799; i=-20.65836751|-|f=48.2871331745; i=-12.37328029|f=33.9422210989; i=-8.71672105288|f=28.3601455449; i=-13.5554404049|f=29.0904929306; i=-10.4583804396|f=29.5374878342; i=-9.51844262607|-
LC81120272014090|f=163.535015844; i=-28.4644780449|-|f=175.436369355; i=-26.3803216807|-|-|-|-|-|f=102.123118397; i=-16.3157581448|-|-|-|-|-|-|f=431.620544375; i=-63.6344353405|f=408.230734907; i=-61.390740139|f=388.65660741; i=-59.0144629906|f=1547.2438965; i=-218.017240383
LC81120272014106|-|f=175.051381076; i=-28.2066893245|-|-|-|-|-|f=327.879604864; i=-229.361745738|f=32.0368069653; i=-7.23102936654|f=405.096549417; i=-63.2214819873|-|-|-|f=74.3497226378; i=-15.4993281438|f=117.295218765; i=-21.0979279468|f=105.388695748; i=-19.4094843427|f=102.887670929; i=-19.0686474855|f=133.275748545; i=-22.7258536295|f=471.511335069; i=-72.8130591757
LC81120272014346|f=50.5167351188; i=-21.5654308759|-|-|-|f=96.9295372481; i=-34.0616877476|f=85.7093266812; i=-26.9129523501|-|-|-|-|f=1220.43811792; i=-315.497343597|f=154.856462433; i=-47.6395397343|f=380.12887884; i=-131.058706572|f=60.8135589062; i=-23.3647092709|-|-|-|-|-
LC81120272014362|f=63.6055532139; i=-27.1865722891|-|-|f=98.8486719714; i=-35.0591594848|-|-|f=54.1460964044; i=-24.4192320228|-|-|-|-|f=178.678835227; i=-53.7582533744|f=349.746303598; i=-117.519509416|-|-|f=38.5412436359; i=-17.0933691678|f=62.7115586009; i=-26.8164562479|-|-
LC81120272015045|f=29.0099428187; i=-10.4989260664|-|-|f=84.2974969497; i=-26.4129725531|-|-|f=35.261096912; i=-13.5238946647|-|-|-|-|-|-|-|f=27.440792025; i=-9.5354600208|-|f=29.6385811516; i=-10.3750592693|f=33.9275688387; i=-11.543617241|-
LC81120272015077|f=39.2021015877; i=-17.77183017|-|-|-|f=53.1685901434; i=-23.9541118789|f=35.9348430805; i=-13.3546617371|-|-|-|-|-|-|-|f=44.7607741718; i=-13.6618546145|-|-|-|f=34.6016722487; i=-12.9383645722|f=269.655489301; i=-54.2320666525
LC81120272015093|-|-|f=331.050377813; i=-242.015534022|-|-|-|-|-|-|f=404.688845142; i=-287.591320049|-|-|-|-|-|-|-|f=67.6451335053; i=-43.7921195882|-
LC81120272015109|f=60.7100414218; i=-16.0147417423|f=104.290579096; i=-19.4149585129|f=32.5101443891; i=-7.81821861542|-|-|-|-|-|-|-|-|-|-|f=96.1958708105; i=-19.9128590727|f=191.633578653; i=-33.5201451948|f=179.750598788; i=-31.4486023613|f=167.855598277; i=-29.836467606|f=77.5837283973; i=-16.1768560038|f=987.857165423; i=-158.387909748
LC81120272015157|f=1035.18071679; i=-128.381388158|-|f=406.234733601; i=-47.1886957479|-|-|-|-|f=408.789776028; i=-286.844293884|-|-|f=4392.87744308; i=-502.160973641|-|-|-|f=1079.85834975; i=-131.609031038|-|f=721.688975812; i=-89.4078510249|f=944.412374824; i=-113.923275486|-
LC81120272015189|f=3494.53398776; i=-337.316075919|-|-|f=1304.15916561; i=-139.167769469|-|-|-|-|-|f=7562.99402867; i=-783.787387693|-|f=872.412610714; i=-96.2565262299|-|-|-|-|-|-|-
LC81120272015333|f=68.093646362; i=-21.0905817673|-|-|f=157.209887998; i=-45.8713824372|f=175.696701022; i=-51.645878369|-|-|-|-|-|f=956.209922748; i=-212.175285813|-|f=230.228623388; i=-69.3075963055|f=97.260610411; i=-27.0220801877|-|-|-|-|-
LC81120272015349|-|-|-|f=386.806262604; i=-133.662604501|f=356.285441142; i=-120.950377603|-|-|-|-|-|-|f=234.357406535; i=-71.0227313942|-|-|-|-|-|-|-
LC81120272015365|f=49.2444835736; i=-12.6763732198|-|f=75.4696629155; i=-15.8978954237|f=59.8232440067; i=-20.5015532426|-|-|f=45.5453764308; i=-13.8398242237|-|f=97.8466944313; i=-20.419303055|-|-|f=95.5574013889; i=-26.4381054496|-|-|-|f=85.9007043954; i=-20.1459686185|-|-|f=348.180278256; i=-68.9051860694
LC81120272016032|f=34.5868493928; i=-8.91903946756|-|f=119.349310118; i=-21.7531183922|-|-|f=27.8387823701; i=-8.51752900083|-|-|f=195.464416693; i=-34.417421172|f=1159.64382758; i=-200.445095362|-|-|-|-|-|-|-|f=31.4927380777; i=-9.48201116925|f=634.308306637; i=-105.493900125
LC81120272016048|f=27.895379328; i=-13.3458420977|f=447.375136908; i=-81.8843550891|f=107.176958474; i=-20.0894513257|-|f=37.7973114735; i=-16.7339400959|-|-|-|f=183.151282389; i=-32.4955657763|-|-|-|-|f=84.5728481056; i=-19.8082419086|-|-|f=35.2866877745; i=-9.01601665407|f=24.6670400578; i=-7.72734683239|f=409.256224599; i=-76.7229319875
LC81120272016064|f=28.4352469275; i=-10.13859846|f=410.274742929; i=-73.1195055859|f=104.763509211; i=-19.7163574053|-|f=61.4854394967; i=-26.2429167096|f=29.7724958749; i=-9.0104919125|-|-|f=171.060632259; i=-30.6866406511|f=761.027147305; i=-134.969901347|-|-|-|-|-|f=35.9422725646; i=-9.10726864729|-|f=32.0598231268; i=-9.55919915002|f=473.62547028; i=-83.1218744393
LC81120272016080|f=30.1290529448; i=-9.68002136394|f=389.275348336; i=-66.2115916116|f=135.635299112; i=-23.4803281565|-|-|f=34.3999481832; i=-9.80797866767|f=33.9597407326; i=-12.6397989311|f=68.6980530842; i=-40.653926673|f=79.4747626853; i=-16.5112364381|f=1002.68290872; i=-165.823358056|-|-|-|-|f=32.1072514748; i=-9.44998079616|f=25.0080504897; i=-7.4685525508|f=32.5408376616; i=-9.42802083279|-|-
LC81120272016096|-|f=1570.37615845; i=-242.823918401|f=480.33972724; i=-75.4359530339|-|-|-|f=264.992222435; i=-52.6940072354|-|f=997.476618888; i=-153.549005504|-|-|-|-|f=349.613512144; i=-63.3932067007|f=646.640513632; i=-109.527500292|f=414.054075026; i=-74.6658192888|f=482.446860743; i=-83.9421063272|-|-

Как мы видим, величины полученных коэффициентов далеки от того, чтобы быть близкими. Например, нормирующие множители одного и того же растра могут отличаться даже не в десятки, а в сотни раз! Как следствие, **в имеющемся виде данная процедура нормирования неприменима** и нужно понять, что с ней не так.

В следующей таблице собрана статистика по нормирующему множителю (factor) для каждой сцены, полученная на основе предыдущей таблицы. Там собрано:
 * Число примеров - сколько раз алгоритм сошелся, т.е. сколько примеров учавствовало в вычислении статистики;
 * Отношение третьего квартиля к первому -- характеристика величины разброса нормирующего множителя для сцены;
 * Полученные минимальное, максимальное значения для нормирующего множителя и квартили.

Строки таблицы отсортированы по отношению квартилей.

Растр |Число примеров|Отношение квартилей (p75% / p25%)|Мин|1-й квартиль p25%|медиана p50%|3-й квартиль p75%|Max
------|-----|---------|---|---|---|---|---
LC81120272016032LGN01|7|12.5571716636|27.8387823701|33.0397937353|119.349310118|414.886361665|1159.64382758
LC81120272016064LGN00|10|10.6105538756|28.4352469275|33.0304354863|83.1244743539|350.471215261|761.027147305
LC81120272016048LGN00|9|5.1903789769|24.6670400578|35.2866877745|84.5728481056|183.151282389|447.375136908
LC81120272015189LGN00|4|3.77158003347|872.412610714|1196.22252689|2399.34657669|4511.64899799|7562.99402867
LC81120272014346LGN00|7|3.65120669804|50.5167351188|73.2614427937|96.9295372481|267.492670636|1220.43811792
LC81120272016080LGN00|11|3.32740015476|25.0080504897|32.3240445682|34.3999481832|107.555030899|1002.68290872
LC81120272014106LGN00|10|2.79841906647|32.0368069653|103.512927134|125.285483655|289.672548917|471.511335069
LC81120272014090LGN00|7|2.47764653878|102.123118397|169.485692599|388.65660741|419.925639641|1547.2438965
LC81120272014362LGN00|7|2.37491935968|38.5412436359|58.4288275027|63.6055532139|138.763753599|349.746303598
LC81120272015109LGN00|9|2.3168595078|32.5101443891|77.5837283973|104.290579096|179.750598788|987.857165423
LC81120272014074LGN00|14|2.14626451554|28.3601455449|30.6453948519|48.9344605434|65.7731235353|3476.95458396
LC81120272015333LGN00|6|1.92961815124|68.093646362|112.247929808|166.45329451|216.595642796|956.209922748
LC81120272015157LGN00|7|1.87092332615|406.234733601|565.23937592|944.412374824|1057.51953327|4392.87744308
LC81120272015093LGN00|3|1.84536620571|67.6451335053|199.347755659|331.050377813|367.869611477|404.688845142
LC81120272016096LGN00|8|1.84535929974|264.992222435|397.943934306|481.393293992|734.349539946|1570.37615845
LC81120272015365LGN00|8|1.68121993467|45.5453764308|57.1785538984|80.6851836554|96.1297246495|348.180278256
LC81120272015077LGN00|6|1.38950565325|34.6016722487|36.7516577073|41.9814378797|51.0666361505|269.655489301
LC81120272015349LGN00|3|1.25810666576|234.357406535|295.321423838|356.285441142|371.545851873|386.806262604
LC81120272015045LGN00|6|1.1975037634|27.440792025|29.1671024019|31.7830749952|34.9277148937|84.2974969497

#### Параметры нормализации внутри маски лесов (forest.mask.30)

Построим аналогичные таблицы для нормировочных коэффициентов но внутри маски лесов (они даны ниже). В целом видно, что ситуация несколько улучшилась. Кроме того, из таблиц видно, что 

 * наибольший разброс коэффициентов приходится на растры LC81120272016032 и LC81120272016048;
 * наибольшие нормирующие множители приходятся на летние и весенние/раннеосенние снимки;
 * разброс коэффициентов для летних снимков невелик, поскольку в большинстве случаев для этих сцен функция поиска PIF не смогла установить инвариантные пиксели.
 
Поэтому попробуем проделать еще один шаг поиска нормировочных коэффициентов, исключив из рассмотрения все летние и весенние снимки.

|LC81120272014074|LC81120272014090|LC81120272014106|LC81120272014346|LC81120272014362|LC81120272015045|LC81120272015077|LC81120272015093|LC81120272015109|LC81120272015157|LC81120272015189|LC81120272015333|LC81120272015349|LC81120272015365|LC81120272016032|LC81120272016048|LC81120272016064|LC81120272016080|LC81120272016096
-|--|--|--|--|--|--|--|--|--|--|--|--|--|--|--|--|--|--|-
LC81120272014074|-|f=386.860055325; i=-67.6688423941|-|-|-|f=62.4720311935; i=-15.7301126046|f=51.485665529; i=-17.2217155208|-|f=104.390717695; i=-24.8990542894|-|-|-|-|-|-|f=32.8818690437; i=-11.1342154508|-|f=67.852035637; i=-14.8978750897|-
LC81120272014090|f=390.542978428; i=-58.9642421458|-|f=459.806926655; i=-62.9220880906|-|-|f=1135.51781394; i=-173.822477811|-|-|f=286.839534484; i=-39.9342575028|f=9086.83125336; i=-1225.52021997|-|-|-|-|f=1594.5406167; i=-228.713500138|-|-|f=1124.73141713; i=-157.995690325|f=2038.00918059; i=-279.498962864
LC81120272014106|-|f=455.937976169; i=-66.886910224|-|-|-|-|-|f=631.937525251; i=-444.820125459|f=57.9436378044; i=-10.4901035039|-|-|-|-|f=225.004693546; i=-40.05185103|f=428.646237477; i=-71.1087365785|-|f=221.179642663; i=-38.3109961279|f=343.75103107; i=-54.6381070756|f=451.992040306; i=-68.8141705447
LC81120272014346|-|-|-|-|f=311.986458985; i=-98.6042723443|-|-|-|-|-|-|-|f=647.90884137; i=-218.809532716|f=159.988308074; i=-54.1969677894|-|-|-|-|-
LC81120272014362|-|-|-|f=316.461620601; i=-101.396393431|-|-|-|-|-|-|-|f=519.900145886; i=-144.550857419|f=571.066481461; i=-186.152540264|-|-|-|-|-|-
LC81120272015045|f=61.6060714383; i=-16.3641365972|f=1200.05056491; i=-306.047469303|-|-|-|-|f=73.0234148639; i=-22.2953392727|-|-|-|-|-|-|-|f=99.9799721149; i=-21.9614032754|-|f=88.0436661058; i=-22.7925313372|f=105.256047275; i=-26.0896420614|-
LC81120272015077|f=50.7232468953; i=-17.5507586381|-|-|-|-|f=74.6225273836; i=-21.4732538041|-|-|-|-|-|-|-|-|f=63.3225302698; i=-16.1468538338|-|f=78.4975548935; i=-19.2662125396|f=67.7887902136; i=-17.7777561985|f=346.47529383; i=-68.4013875932
LC81120272015093|-|-|f=635.498215189; i=-466.470723247|-|-|-|-|-|-|f=611.260226384; i=-436.377285829|-|-|-|-|-|-|-|-|-
LC81120272015109|f=102.638100464; i=-25.009306342|f=294.935177015; i=-49.3857183977|f=59.0326170576; i=-10.9409880548|-|-|-|-|-|-|-|-|-|-|-|f=420.304685813; i=-71.8169371156|f=521.027190839; i=-86.7766917058|f=541.878009668; i=-90.0530509713|f=361.683521973; i=-60.2427528711|f=1917.57561312; i=-301.826359846
LC81120272015157|-|f=9483.03996223; i=-1030.94380187|-|-|-|-|-|f=607.790731143; i=-428.412743937|-|-|-|-|-|-|-|f=2554.9509222; i=-283.075995474|-|-|-
LC81120272015189|-|-|-|-|-|-|-|-|-|-|-|f=1937.66638164; i=-199.64627761|-|-|-|-|-|-|-
LC81120272015333|-|-|-|-|f=530.192241266; i=-145.377934013|-|-|-|-|-|f=2028.07237325; i=-432.376598515|-|f=708.535478056; i=-198.216691554|-|-|-|-|-|-
LC81120272015349|-|-|-|f=659.096339316; i=-223.768504509|f=582.030566021; i=-194.913974454|-|-|-|-|-|-|f=698.834819778; i=-212.928716258|-|-|-|-|-|-|-
LC81120272015365|-|-|f=229.444866752; i=-41.3953728171|f=159.01697096; i=-46.4248464366|-|-|-|-|-|-|-|-|-|-|-|f=175.151608033; i=-39.7178720412|-|-|f=737.40794622; i=-129.77158193
LC81120272016032|-|f=1625.28599074; i=-266.513257176|f=437.138690033; i=-73.3990931795|-|-|f=100.844461203; i=-19.332885732|f=64.2218842816; i=-14.6744498942|-|f=427.829233215; i=-73.5855308139|-|-|-|-|-|-|-|-|f=144.727961712; i=-26.6524783703|f=1298.76207087; i=-209.682520053
LC81120272016048|f=32.2849981924; i=-10.8966324614|-|-|-|-|-|-|-|f=529.577870929; i=-89.1756662536|f=2937.429941; i=-523.399557372|-|-|-|f=172.552753517; i=-39.1100180536|-|-|f=156.107120616; i=-28.6694872328|f=108.001182208; i=-20.9478996879|f=895.402607871; i=-162.389421803
LC81120272016064|-|-|f=225.318035207; i=-39.5759664324|-|-|f=88.8905477623; i=-18.9220277489|f=80.2620604512; i=-17.46461223|-|f=553.609504817; i=-92.6957296107|-|-|-|-|-|-|f=158.910433966; i=-28.7819462089|-|f=138.775014841; i=-25.6341335659|f=841.574213923; i=-144.181038038
LC81120272016080|f=69.1023002853; i=-15.2016575328|f=1119.48356806; i=-177.752875674|f=350.549104989; i=-56.5306314187|-|-|f=103.612995561; i=-20.4065880271|f=68.4958068262; i=-15.1420103541|-|f=362.930656887; i=-62.1188086969|-|-|-|-|-|f=144.999620561; i=-25.8693648881|f=109.440039848; i=-19.675916549|f=141.825731876; i=-25.2619125242|-|-
LC81120272016096|-|f=2353.13728478; i=-372.047985811|f=460.809421099; i=-71.4448778721|-|-|-|f=340.112962232; i=-66.3468188249|-|f=1950.86650584; i=-302.041229048|-|-|-|-|f=724.435343027; i=-126.097677011|f=1329.29787533; i=-219.162211718|f=897.232973013; i=-156.266067775|f=862.167599716; i=-146.264946927|-|-

Растр|Число примеров|Отношение квартилей (p75% / p25%)|Мин|1-й квартиль p25%|медиана p50%|3-й квартиль p75%|Max
-|--|--|--|--|--|--|--|-
LC81120272016032|7|7.06879355709|64.2218842816|122.786211457|427.829233215|867.95038045|1625.28599074
LC81120272016048|7|5.39543991448|32.2849981924|132.054151412|172.552753517|712.4902394|2937.429941
LC81120272014090|8|3.85410774562|286.839534484|442.490939599|1130.12461554|1705.40775767|9086.83125336
LC81120272015157|3|3.8061884921|607.790731143|1581.37082667|2554.9509222|6018.99544221|9483.03996223
LC81120272016064|7|3.4213674265|80.2620604512|113.832781301|158.910433966|389.463770012|841.574213923
LC81120272016080|9|3.38325422492|68.4958068262|103.612995561|141.825731876|350.549104989|1119.48356806
LC81120272016096|8|2.25455574904|340.112962232|658.528862545|879.700286365|1484.69003296|2353.13728478\
LC81120272015333|3|2.209208536|530.192241266|619.363859661|708.535478056|1368.30392565|2028.07237325
LC81120272015109|8|2.13172632342|59.0326170576|246.860907877|390.994103893|526.239895546|1917.57561312
LC81120272015365|4|2.08298217219|159.01697096|171.117948765|202.298237392|356.435636619|737.40794622
LC81120272014346|3|2.03378520919|159.988308074|235.987383529|311.986458985|479.947650177|647.90884137
LC81120272014106|8|2.02178842585|57.9436378044|224.048430826|386.198634273|452.978524272|631.937525251
LC81120272014074|6|1.75644630237|32.8818690437|54.2322569451|65.1620334153|95.2560471805|386.860055325
LC81120272015045|6|1.35372609139|61.6060714383|76.7784776744|94.0118191104|103.937028485|1200.05056491
LC81120272014362|3|1.30441953597|316.461620601|418.180883243|519.900145886|545.483313673|571.066481461
LC81120272015077|6|1.20313293829|50.7232468953|64.4390952558|71.2056587986|77.528798016|346.47529383
LC81120272015349|3|1.09411145086|582.030566021|620.563452668|659.096339316|678.965579547|698.834819778
LC81120272015093|2|1.01963163324|611.260226384|617.319723586|623.379220787|629.438717988|635.498215189
LC81120272015189|1|1.0|1937.66638164|1937.66638164|1937.66638164|1937.66638164|1937.66638164

#### Нормировка только зимних снимков

Оставим в рассмотрении только снимки, снятые в зимний период (эти растры помечены "зимними" в [блокноте](020_RasterSelection.ipynb)).

In [ ]:
winter_rasters = ['clean.topo.toar_LC81120272014074LGN00_B1@landsat', 
                  'clean.topo.toar_LC81120272014346LGN00_B1@landsat', 
                  'clean.topo.toar_LC81120272014362LGN00_B1@landsat', 
                  'clean.topo.toar_LC81120272015045LGN00_B1@landsat', 
                  'clean.topo.toar_LC81120272015077LGN00_B1@landsat', 
                  'clean.topo.toar_LC81120272015333LGN00_B1@landsat', 
                  'clean.topo.toar_LC81120272015349LGN00_B1@landsat',
                  'clean.topo.toar_LC81120272015365LGN00_B1@landsat',
                  'clean.topo.toar_LC81120272016032LGN01_B1@landsat', 
                  'clean.topo.toar_LC81120272016048LGN00_B1@landsat', 
                  'clean.topo.toar_LC81120272016064LGN00_B1@landsat', 
                  'clean.topo.toar_LC81120272016080LGN00_B1@landsat'
                 ]

|LC81120272014074|LC81120272014346|LC81120272014362|LC81120272015045|LC81120272015077|LC81120272015333|LC81120272015349|LC81120272015365|LC81120272016032|LC81120272016048|LC81120272016064|LC81120272016080
-|--|--|--|--|--|--|--|--|--|--|--|-
LC81120272014074|-|-|-|f=62.4720311935; i=-15.7301126046|f=51.485665529; i=-17.2217155208|-|-|-|-|f=32.8818690437; i=-11.1342154508|-|f=67.852035637; i=-14.8978750897
LC81120272014346|-|-|f=311.986458985; i=-98.6042723443|-|-|-|f=647.90884137; i=-218.809532716|f=159.988308074; i=-54.1969677894|-|-|-|-
LC81120272014362|-|f=316.461620601; i=-101.396393431|-|-|-|f=519.900145886; i=-144.550857419|f=571.066481461; i=-186.152540264|-|-|-|-|-
LC81120272015045|f=61.6060714383; i=-16.3641365972|-|-|-|f=73.0234148639; i=-22.2953392727|-|-|-|f=99.9799721149; i=-21.9614032754|-|f=88.0436661058; i=-22.7925313372|f=105.256047275; i=-26.0896420614
LC81120272015077|f=50.7232468953; i=-17.5507586381|-|-|f=74.6225273836; i=-21.4732538041|-|-|-|-|f=63.3225302698; i=-16.1468538338|-|f=78.4975548935; i=-19.2662125396|f=67.7887902136; i=-17.7777561985
LC81120272015333|-|-|f=530.192241266; i=-145.377934013|-|-|-|f=708.535478056; i=-198.216691554|-|-|-|-|-
LC81120272015349|-|f=659.096339316; i=-223.768504509|f=582.030566021; i=-194.913974454|-|-|f=698.834819778; i=-212.928716258|-|-|-|-|-|-
LC81120272015365|-|f=159.01697096; i=-46.4248464366|-|-|-|-|-|-|-|f=175.151608033; i=-39.7178720412|-|-
LC81120272016032|-|-|-|f=100.844461203; i=-19.332885732|f=64.2218842816; i=-14.6744498942|-|-|-|-|-|-|f=144.727961712; i=-26.6524783703
LC81120272016048|f=32.2849981924; i=-10.8966324614|-|-|-|-|-|-|f=172.552753517; i=-39.1100180536|-|-|f=156.107120616; i=-28.6694872328|f=108.001182208; i=-20.9478996879
LC81120272016064|-|-|-|f=88.8905477623; i=-18.9220277489|f=80.2620604512; i=-17.46461223|-|-|-|-|f=158.910433966; i=-28.7819462089|-|f=138.775014841; i=-25.6341335659
LC81120272016080|f=69.1023002853; i=-15.2016575328|-|-|f=103.612995561; i=-20.4065880271|f=68.4958068262; i=-15.1420103541|-|-|-|f=144.999620561; i=-25.8693648881|f=109.440039848; i=-19.675916549|f=141.825731876; i=-25.2619125242|-

Растр|Число примеров|Отношение квартилей (p75% / p25%)|Мин|1-й квартиль p25%|медиана p50%|3-й квартиль p75%|Max
-|--|--|--|--|--|--|--|-
LC81120272014346|3|2.03378520919|159.988308074|235.987383529|311.986458985|479.947650177|647.90884137
LC81120272016048|4|1.79875026769|32.2849981924|89.0721362041|132.054151412|160.218528841|172.552753517
LC81120272016080|6|1.72043423931|68.4958068262|77.7299741042|106.526517705|133.729308869|144.999620561
LC81120272016064|4|1.65805591181|80.2620604512|86.7334259345|113.832781301|143.808869622|158.910433966
LC81120272016032|3|1.48771951178|64.2218842816|82.5331727422|100.844461203|122.786211457|144.727961712
LC81120272015045|5|1.36914950227|61.6060714383|73.0234148639|88.0436661058|99.9799721149|105.256047275
LC81120272014074|4|1.36260101906|32.8818690437|46.8347164077|56.9788483613|63.8170323044|67.852035637
LC81120272014362|3|1.30441953597|316.461620601|418.180883243|519.900145886|545.483313673|571.066481461
LC81120272015077|5|1.17845144636|50.7232468953|63.3225302698|67.7887902136|74.6225273836|78.4975548935
LC81120272015333|2|1.15514095975|530.192241266|574.778050464|619.363859661|663.949668859|708.535478056
LC81120272015349|3|1.09411145086|582.030566021|620.563452668|659.096339316|678.965579547|698.834819778
LC81120272015365|2|1.04947738335|159.01697096|163.050630228|167.084289496|171.117948765|175.151608033


Числа стали выглядеть лучше, но, если честно, то трудно сказать, почему. То ли действительно исключение всех незимних сцен сработало, то ли разброс уменьшился за счет уменьшения числа анализируемых примеров. Вероятно, сработало и то, и другое.

**Вывод.** С некоторой натяжкой будем считать, что в превом приближении мы нашли сцены, для которых нормирующие коэффициенты будут более-менее устойчивыми.

#### Влияние  regression_eps на устойчивость.

Попробуем поменять параметр regression_eps (основной параметр, отвечающий за точность), в широком диапазоне и посмотрим на то, как поменяется таблица с коэффициентами:
 
 * берем зимние растры и маску лесов;
 * для каждого regression_eps = 0.3, 0.15, 0.01 рассчитываем нормирующие коэффициенты.
 
Таблицы с результами приводятся ниже. Выводы по таблицам:

 * При больших значениях regression_eps (чем меньше этот параметр, тем более жеские требования к PIF, ожидается, что точность нормирующего множителя будет выше) отношение квартилей становится меньше. Этот результат несколько неожиданен. Возможно, он достигается за счет того, что чем выше значение regression_eps, тем больше пикселей используется при расчетах и тем ближе среняя яркостей выбранных пикселей к среднем по растру.
 * Разброс медиан остается достаточно большим даже для высоких значений regression_eps. Например, для растра LC81120272015349: 277.2 (eps=0.3), 236.0 (eps=0.15), 659.0 (eps=0.01), 641.9 (eps=0.001).

|LC81120272014074|LC81120272014346|LC81120272014362|LC81120272015045|LC81120272015077|LC81120272015333|LC81120272015349|LC81120272015365|LC81120272016032|LC81120272016048|LC81120272016064|LC81120272016080
-|--|--|--|--|--|--|--|--|--|--|--|-
LC81120272014074|-|f=50.9402315987; i=-14.7771707153|f=91.1460923644; i=-30.1504299889|f=62.1375898576; i=-15.8901886398|f=54.5679743702; i=-18.4886877428|f=149.931000944; i=-37.6193144521|f=168.916607968; i=-56.7977477318|f=76.5783328816; i=-16.2567381406|f=79.2915579584; i=-16.2237996129|f=33.1946535244; i=-11.7639889593|f=63.250120961; i=-14.8279828397|f=58.2979869385; i=-13.7223843168
LC81120272014346|f=84.5260898415; i=-25.7600719745|-|f=311.986458985; i=-98.6042723443|f=184.042034675; i=-60.1290530107|f=154.551551431; i=-52.800092315|f=593.768127109; i=-177.929944756|f=455.197675247; i=-141.280463335|f=193.825282274; i=-66.0912707284|f=135.093279692; i=-43.8718934289|f=69.2047408023; i=-25.8255383555|f=124.129061315; i=-40.0677585924|f=143.366411066; i=-45.7034477902
LC81120272014362|f=140.173372969; i=-48.433836019|f=316.461620601; i=-101.396393431|-|f=232.878946185; i=-72.8960446533|f=156.301956471; i=-52.3250842905|f=525.533216663; i=-169.328266589|f=554.807568314; i=-175.295377494|f=279.237211365; i=-89.6654030179|f=229.861684977; i=-66.6584114694|f=114.745513326; i=-35.8488697644|f=195.950008055; i=-58.2251151689|f=181.892091641; i=-56.5809103539
LC81120272015045|f=51.454851256; i=-14.8342888115|f=91.5986998771; i=-27.6010768984|f=118.508594437; i=-30.8556891496|-|f=68.7112516019; i=-20.774503799|f=298.59474878; i=-73.9647376428|f=281.51167807; i=-80.1235685002|f=73.0840185868; i=-20.2777105502|f=52.3363831475; i=-14.8220663434|f=49.2261461854; i=-14.2878232291|f=88.0436661058; i=-22.7925313372|f=96.3488012548; i=-23.9763125253
LC81120272015077|f=48.4089474803; i=-18.1083258084|f=76.3907410357; i=-24.0424653358|f=78.8279634014; i=-23.4094370404|f=72.3702460301; i=-20.7779163802|-|f=259.315222408; i=-64.8084414011|f=139.725413808; i=-46.5245839128|f=83.9560955012; i=-25.0281988399|f=60.8839483369; i=-16.5486159801|f=66.0151888872; i=-15.8159621398|f=78.4975548935; i=-19.2662125396|f=62.5250195481; i=-17.0816142278
LC81120272015333|f=244.365234648; i=-63.2012869249|f=993.041614123; i=-267.991566434|f=813.559482326; i=-217.812731895|f=498.021100306; i=-124.128386529|f=421.103867476; i=-108.634711334|-|f=1280.57457965; i=-334.101514266|f=385.70375382; i=-107.803734072|f=191.671849494; i=-51.3648928936|f=179.928348078; i=-46.862001273|f=289.365572783; i=-68.0228815181|f=262.138139388; i=-64.2857448003
LC81120272015349|f=277.196122916; i=-96.8622126028|f=755.747512304; i=-261.80386937|f=795.366618833; i=-278.393293829|f=471.017769415; i=-147.078999479|f=233.725859567; i=-81.2904007499|f=734.989785494; i=-255.983156417|-|f=441.401517786; i=-137.983751256|f=207.543367581; i=-66.2479928758|f=231.926889371; i=-75.5112645136|f=222.78748117; i=-75.123952075|f=252.816763283; i=-83.0305415937
LC81120272015365|f=127.557102566; i=-27.4502291077|f=138.414997813; i=-36.6350519112|f=178.241314737; i=-42.5565424969|f=122.786860846; i=-29.3519849267|f=138.570509943; i=-36.8770804506|f=210.23886759; i=-54.1812647115|f=220.005243311; i=-50.148909442|-|f=120.414504829; i=-27.3087897962|f=62.4769434178; i=-19.717293054|f=137.904429843; i=-30.0631134013|f=161.625749333; i=-35.6895451619
LC81120272016032|f=95.0093238899; i=-18.7752857959|f=67.0634220876; i=-16.3937657119|f=117.539586659; i=-24.521858144|f=62.2196520328; i=-14.4410409457|f=69.8142909936; i=-16.4590953986|f=96.5802653582; i=-22.128152226|f=105.368776953; i=-29.2317093941|f=82.7745896363; i=-17.8258790492|-|f=41.6941497372; i=-11.3365812163|f=158.095652572; i=-29.0797237771|f=144.727961712; i=-26.6524783703
LC81120272016048|f=30.5521099051; i=-11.5887682162|f=35.7459982932; i=-12.8467015569|f=56.5210816771; i=-14.8582019444|f=47.4463659514; i=-12.1290787095|f=57.7743248485; i=-12.7646382106|f=94.0997947221; i=-21.3182149642|f=121.204533438; i=-36.5601193029|f=34.8199431313; i=-12.3772158469|f=32.7650742939; i=-9.58166926155|-|f=96.837122445; i=-19.0069690326|f=88.2651181086; i=-17.9107916036
LC81120272016064|f=66.9981797817; i=-15.504009633|f=65.1237916667; i=-18.7643614956|f=97.0588828422; i=-22.4472911454|f=88.8905477623; i=-18.9220277489|f=80.2620604512; i=-17.46461223|f=146.45891038; i=-29.4291174062|f=114.864366615; i=-36.1926430683|f=79.744827968; i=-16.9762158162|f=132.432289817; i=-24.5384620913|f=108.387762395; i=-20.6769151627|-|f=135.756775782; i=-25.2015014934
LC81120272016080|f=66.7315767289; i=-15.3112580572|f=70.6946859925; i=-17.8451085174|f=91.4504056855; i=-23.5288941863|f=115.031625707; i=-22.2909103461|f=67.4389322494; i=-15.2756626451|f=138.908479185; i=-29.4082762484|f=131.552484429; i=-38.8120351405|f=107.657984489; i=-21.6542004176|f=144.999620561; i=-25.8693648881|f=112.718879257; i=-20.4050326793|f=139.63490614; i=-24.9898963237|-

|LC81120272014074|LC81120272014346|LC81120272014362|LC81120272015045|LC81120272015077|LC81120272015333|LC81120272015349|LC81120272015365|LC81120272016032|LC81120272016048|LC81120272016064|LC81120272016080
-|--|--|--|--|--|--|--|--|--|--|--|-
LC81120272014074|-|f=62.4527841568; i=-20.749432594|f=105.829722986; i=-35.7459094108|f=62.1375898576; i=-15.8901886398|f=54.5679743702; i=-18.4886877428|f=159.230289567; i=-41.2037478314|f=212.217167391; i=-72.9449933661|f=110.474854343; i=-22.0368402451|f=79.2915579584; i=-16.2237996129|f=33.1946535244; i=-11.7639889593|f=63.250120961; i=-14.8279828397|f=58.2979869385; i=-13.7223843168
LC81120272014346|f=79.272902158; i=-26.2457669191|-|f=311.986458985; i=-98.6042723443|f=166.168726922; i=-48.1336395145|f=118.029198746; i=-41.4461491688|f=631.634036582; i=-182.641112851|f=523.30798213; i=-170.655427268|f=190.445344238; i=-64.9627222885|f=82.8063674965; i=-29.689373313|f=58.2487015094; i=-23.1050405305|f=84.25265432; i=-27.4263177086|f=74.787979306; i=-22.6133221436
LC81120272014362|f=133.459180081; i=-46.0031858632|f=316.461620601; i=-101.396393431|-|f=192.444733552; i=-60.2576949922|f=109.518670047; i=-39.530708557|f=533.808608026; i=-169.325173892|f=567.681427175; i=-179.887840899|f=253.601571152; i=-79.3421960932|f=85.43361239; i=-28.924061644|f=70.8016418143; i=-24.3882559358|f=101.32738299; i=-33.802996241|f=114.071574532; i=-37.3604425287
LC81120272015045|f=51.454851256; i=-14.8342888115|f=123.088639819; i=-33.6655413639|f=143.41019531; i=-41.9417787766|-|f=68.7112516019; i=-20.774503799|f=396.345286144; i=-94.262080939|f=331.954004629; i=-95.8984573562|f=124.369892058; i=-31.4970966139|f=52.3363831475; i=-14.8220663434|f=49.2261461854; i=-14.2878232291|f=88.0436661058; i=-22.7925313372|f=96.3488012548; i=-23.9763125253
LC81120272015077|f=48.4089474803; i=-18.1083258084|f=86.5594957869; i=-29.457269783|f=82.1049912754; i=-28.6274714649|f=72.3702460301; i=-20.7779163802|-|f=220.541251282; i=-54.7738882137|f=177.607190894; i=-61.8026294966|f=97.3019367087; i=-26.3141108053|f=60.8839483369; i=-16.5486159801|f=66.0151888872; i=-15.8159621398|f=78.4975548935; i=-19.2662125396|f=62.5250195481; i=-17.0816142278
LC81120272015333|f=209.143113921; i=-55.0366813913|f=827.926682848; i=-224.40821943|f=718.27264769; i=-194.376254984|f=512.349537138; i=-122.762080346|f=286.30422653; i=-72.2348556284|-|f=990.399693253; i=-267.327719899|f=307.84743156; i=-80.7287329527|f=204.936035315; i=-47.7336848134|f=224.982060639; i=-52.3650166959|f=267.805637408; i=-59.4965803079|f=225.045131932; i=-58.242879046
LC81120272015349|f=276.752812115; i=-96.8275561155|f=685.620929484; i=-233.946295591|f=767.597036488; i=-267.752201574|f=438.18515087; i=-132.756097767|f=225.82807865; i=-79.905631331|f=771.230709108; i=-254.509166553|-|f=235.972118373; i=-74.7511132851|f=125.399138839; i=-40.5578695133|f=223.189848584; i=-73.2862665512|f=196.575587541; i=-68.2722850599|f=191.362098627; i=-63.5246262874
LC81120272015365|f=140.723473135; i=-29.2760901019|f=143.558481635; i=-38.6788314998|f=188.717078732; i=-48.7281522576|f=163.301467348; i=-37.1388020578|f=126.067254146; i=-32.1104444898|f=228.666539168; i=-56.4441838571|f=172.54172675; i=-50.1697107736|-|f=161.943283907; i=-35.1267158893|f=235.658195897; i=-53.8000115626|f=243.891905914; i=-47.9703190103|f=162.596525899; i=-35.2732533281
LC81120272016032|f=95.0093238899; i=-18.7752857959|f=60.0849195375; i=-19.7278186824|f=64.8619291889; i=-20.4411911016|f=62.2196520328; i=-14.4410409457|f=69.8142909936; i=-16.4590953986|f=148.69821904; i=-31.5708336236|f=93.8656090054; i=-28.8011641284|f=117.874659323; i=-24.0693636348|-|f=41.6941497372; i=-11.3365812163|f=158.095652572; i=-29.0797237771|f=144.727961712; i=-26.6524783703
LC81120272016048|f=30.5521099051; i=-11.5887682162|f=42.4747998217; i=-17.0324772041|f=52.5099219166; i=-17.3991307536|f=47.4463659514; i=-12.1290787095|f=57.7743248485; i=-12.7646382106|f=164.984832543; i=-34.9583119798|f=167.59916305; i=-53.5511233783|f=174.449921419; i=-38.7220939067|f=32.7650742939; i=-9.58166926155|-|f=96.837122445; i=-19.0069690326|f=88.2651181086; i=-17.9107916036
LC81120272016064|f=66.9981797817; i=-15.504009633|f=62.998964026; i=-19.9386130943|f=75.6911892591; i=-23.7608005752|f=88.8905477623; i=-18.9220277489|f=80.2620604512; i=-17.46461223|f=200.323833354; i=-41.8140461436|f=145.759897451; i=-49.7490890442|f=180.762975107; i=-33.2526064882|f=132.432289817; i=-24.5384620913|f=108.387762395; i=-20.6769151627|-|f=135.756775782; i=-25.2015014934
LC81120272016080|f=66.7315767289; i=-15.3112580572|f=55.3982230644; i=-15.528873975|f=85.0362377246; i=-26.148684238|f=115.031625707; i=-22.2909103461|f=67.4389322494; i=-15.2756626451|f=168.357330204; i=-41.7488515925|f=141.44859837; i=-45.2453469819|f=118.347440873; i=-23.5913716221|f=144.999620561; i=-25.8693648881|f=112.718879257; i=-20.4050326793|f=139.63490614; i=-24.9898963237|-

|LC81120272014074|LC81120272014346|LC81120272014362|LC81120272015045|LC81120272015077|LC81120272015333|LC81120272015349|LC81120272015365|LC81120272016032|LC81120272016048|LC81120272016064|LC81120272016080
-|--|--|--|--|--|--|--|--|--|--|--|-
LC81120272014074|-|-|-|f=65.3004186403; i=-16.3020736259|f=49.515640026; i=-16.444723877|-|-|f=122.052488969; i=-25.6178894804|f=106.069460561; i=-20.2231844856|f=31.2673201352; i=-10.3474921911|f=67.2219281565; i=-15.3711535131|f=67.881380654; i=-14.9232741356
LC81120272014346|-|-|f=311.088191521; i=-98.2375771863|-|-|f=545.308613853; i=-149.290636396|f=641.661538363; i=-216.888163345|f=168.464750699; i=-50.6669294672|-|-|-|-
LC81120272014362|-|f=315.798739608; i=-101.18642491|-|-|-|f=442.171769094; i=-121.751211211|f=552.852676446; i=-180.879382768|f=154.396648966; i=-45.7277950647|-|-|-|-
LC81120272015045|f=65.2198945651; i=-16.9433014195|-|-|-|f=75.9725957243; i=-23.3495931876|-|-|-|-|-|-|-
LC81120272015077|f=49.5852060076; i=-16.6198377153|-|-|f=76.3049265396; i=-22.0399629352|-|-|-|-|f=65.5992289724; i=-16.446954897|-|-|-
LC81120272015333|-|f=546.239897001; i=-149.432621308|f=442.953989288; i=-121.871946032|-|-|-|f=713.840245518; i=-199.996459878|f=278.761153089; i=-70.2350674165|-|-|-|-
LC81120272015349|-|f=641.874800243; i=-217.547035592|f=553.983857358; i=-184.18166318|-|-|f=716.346356879; i=-213.011456807|-|-|-|-|-|-
LC81120272015365|f=122.231190049; i=-25.6662107413|f=168.883686452; i=-46.485487209|f=154.142126684; i=-45.6152704175|-|-|f=278.280107146; i=-70.1176972146|-|-|f=185.7573638; i=-42.0159348154|f=32.1717836749; i=-8.36199689531|f=211.582098973; i=-42.0660969179|f=118.445455468; i=-27.1388894883
LC81120272016032|f=106.258793108; i=-20.2726635273|-|-|-|f=65.7708034826; i=-14.8473921482|-|-|f=185.413179742; i=-41.8899886595|-|f=498.663176153; i=-86.2979073775|f=299.559623037; i=-50.9951985856|f=121.948831533; i=-22.7203695738
LC81120272016048|f=31.2260488343; i=-10.329093442|-|-|-|-|-|-|f=32.166838971; i=-8.36250340708|f=497.756478213; i=-85.9618447282|-|f=184.272208139; i=-33.6389544586|f=109.24936224; i=-21.0858209744
LC81120272016064|f=67.345093147; i=-15.4089267203|-|-|-|-|-|-|f=211.160871226; i=-41.96122413|f=299.007176801; i=-50.8445846835|f=184.558558134; i=-33.5331431791|-|f=144.118481264; i=-26.3693059029
LC81120272016080|f=68.0176809202; i=-14.9549945817|-|-|-|-|-|-|f=118.217878015; i=-27.0602972524|f=121.757294635; i=-22.6615797172|f=109.331963332; i=-19.6395045975|f=144.641410458; i=-25.5550451157|-

Сводные таблицы:

Растр|Число примеров|Отношение квартилей (p75% / p25%)|Мин|1-й квартиль p25%|медиана p50%|3-й квартиль p75%|Max
-|--|--|--|--|--|--|--|-
LC81120272015349|11|2.58992899249|207.543367581|232.826374469|277.196122916|603.003777454|795.366618833
LC81120272015333|11|2.58948044547|179.928348078|253.251687018|385.70375382|655.790291316|1280.57457965
LC81120272016048|11|2.58431913681|30.5521099051|35.2829707123|56.5210816771|91.1824564153|121.204533438
LC81120272014346|11|1.95126600313|69.2047408023|129.611170504|154.551551431|252.90587063|593.768127109
LC81120272015045|11|1.77498218893|49.2261461854|60.5238173747|88.0436661058|107.428697846|298.59474878
LC81120272014362|11|1.76141134148|114.745513326|169.097024056|229.861684977|297.849415983|554.807568314
LC81120272016080|11|1.66801819787|66.7315767289|81.072545839|112.718879257|135.230481807|144.999620561
LC81120272016032|11|1.62852197479|41.6941497372|68.4388565406|95.0093238899|111.454181806|158.095652572
LC81120272016064|11|1.54553756326|65.1237916667|80.0034442096|97.0588828422|123.648328216|146.45891038
LC81120272014074|11|1.51008903257|33.1946535244|56.4329806544|63.250120961|85.2188251614|168.916607968
LC81120272015365|11|1.35760039682|62.4769434178|125.171981706|138.414997813|169.933532035|220.005243311
LC81120272015077|11|1.26640574871|48.4089474803|64.2701042177|76.3907410357|81.3920294513|259.315222408


REGR_EPS = 0.15
MAXITER = 25

Растр|Число примеров|Отношение квартилей (p75% / p25%)|Мин|1-й квартиль p25%|медиана p50%|3-й квартиль p75%|Max
-|--|--|--|--|--|--|--|-
LC81120272014346|11|3.0999140377|58.2487015094|81.0396348272|118.029198746|251.215901611|631.634036582
LC81120272016048|11|2.91168328099|30.5521099051|44.9605828865|57.7743248485|130.910977494|174.449921419
LC81120272014074|11|1.79601894293|33.1946535244|60.2177883981|63.250120961|108.152288664|212.217167391
LC81120272015333|11|2.73455072303|204.936035315|225.013596286|286.30422653|615.311092414|990.399693253
LC81120272014362|11|2.70369392048|70.8016418143|105.423026518|133.459180081|285.031595876|567.681427175
LC81120272015349|11|2.67722395328|125.399138839|209.882718063|235.972118373|561.903040177|771.230709108
LC81120272015045|11|2.21218768894|49.2261461854|60.5238173747|96.3488012548|133.890043684|396.345286144
LC81120272016032|11|2.06640977009|41.6941497372|63.5407906108|93.8656090054|131.301310518|158.095652572
LC81120272016080|11|1.84347067498|55.3982230644|76.237584987|115.031625707|140.541752255|168.357330204
LC81120272016064|11|1.80513502447|62.998964026|77.9766248552|108.387762395|140.758336616|200.323833354
LC81120272015077|11|1.43038069359|48.4089474803|64.2701042177|78.4975548935|91.9307162478|220.541251282
LC81120272015365|11|1.3662232595|126.067254146|152.750882771|163.301467348|208.69180895|243.891905914

REGR_EPS = 0.001
MAXITER = 50

Растр|Число примеров|Отношение квартилей (p75% / p25%)|Мин|1-й квартиль p25%|медиана p50%|3-й квартиль p75%|Max
-|--|--|--|--|--|--|--|-
LC81120272016048|5|5.728638997|31.2260488343|32.166838971|109.24936224|184.272208139|497.756478213
LC81120272016032|6|2.45979132154|65.7708034826|110.181302714|153.681005638|271.023012213|498.663176153
LC81120272014346|4|2.06728397593|168.464750699|275.432331315|428.198402687|569.396844981|641.661538363
LC81120272014362|4|1.70573620385|154.396648966|275.448216948|378.985254351|469.841995932|552.852676446
LC81120272015365|8|1.58481208433|32.1717836749|121.284756404|161.512906568|192.213547594|278.280107146
LC81120272014074|7|1.51503930056|31.2673201352|57.4080293331|67.2219281565|86.9754206075|122.052488969
LC81120272016064|5|1.46518940093|67.345093147|144.118481264|184.558558134|211.160871226|299.007176801
LC81120272015333|4|1.46337776924|278.761153089|401.905780238|494.596943144|588.13998413|713.840245518
LC81120272015077|3|1.23197336113|49.5852060076|57.59221749|65.5992289724|70.952077756|76.3049265396
LC81120272015349|3|1.13577064354|553.983857358|597.929328801|641.874800243|679.110578561|716.346356879
LC81120272016080|5|1.11364774695|68.0176809202|109.331963332|118.217878015|121.757294635|144.641410458
LC81120272015045|2|1.0791710115|65.2198945651|67.9080698549|70.5962451447|73.2844204345|75.9725957243

<a id='Implementation'></a>
## Выводы и реализация

Как было показано в [предыдущем разлеле](#Practic), процедура нормализации довольно неустойчива. Частично это можно исправить за счет увеличения значения параметра regr_eps. Предоположительно это происходит потому, что увеличение параметра приводит к выбору большего числа PIF и, как следствие, к приближению статхарактеристик PIF к статхарактеристикам всего растра. (Но вопрос о том, действительно ли это так, не исследовался).

С очень большими оговорками и учетом того, что нужно двигаться дальше, предлагается следующая процедура нормирования:

 1. Для каждого зимнего растра находим наборы PIFов относительно остальных растров (с использованием маски лесов).
 2. Объединяем все PIFы в один общий растр. Этот объединенный растр PIFов и будет использоваться в качестве основы для рассчета нормирующих коэффициентов.

После нормирования перейдем непосредственно к классификации. В случае, если результаты нас не устроят, нормирование - первый кандидат на дополнительное исследование и модификацию.

Вообще, похоже, что вопросами нормирования стоило бы заняться отдельно (эх, вот еще одно поле для деятельности).

### Выбор PIF

#### Отдельные PIF

Сгенерируем растры PIFов для зимних сцен по каждому каналу:

In [ ]:
# Prefixes for scenes
winter_rasters = ['clean.topo.toar_LC81120272014074LGN00_B', 
                  'clean.topo.toar_LC81120272014346LGN00_B', 
                  'clean.topo.toar_LC81120272014362LGN00_B', 
                  'clean.topo.toar_LC81120272015045LGN00_B', 
                  'clean.topo.toar_LC81120272015077LGN00_B', 
                  'clean.topo.toar_LC81120272015333LGN00_B', 
                  'clean.topo.toar_LC81120272015349LGN00_B',
                  'clean.topo.toar_LC81120272015365LGN00_B',
                  'clean.topo.toar_LC81120272016032LGN01_B', 
                  'clean.topo.toar_LC81120272016048LGN00_B', 
                  'clean.topo.toar_LC81120272016064LGN00_B', 
                  'clean.topo.toar_LC81120272016080LGN00_B'
                 ]

rast1 = 'clean.topo.toar_LC81120272014074LGN00_B1'
grs.grass.run_command('g.region', rast=rast1, res='120')

grs.grass.run_command('r.mask', flags='r')
grs.grass.run_command('r.mapcalc', expression="MASK=forest.mask.30@treecover", overwrite=True)

REGR_EPS = 0.1
MAXITER = 25

for band_num in range(1, 8):
    
    for i in range(len(winter_rasters)):
        
        iter_count = len(winter_rasters) - 1 
        rast1 = winter_rasters[i]
        rast1 += str(band_num)
        grs.grass.run_command('g.region', rast=rast1, res=120)
        
        with tqdm_notebook(total=iter_count, desc=rast1) as pbar:
            pif_count = 0  # count of rasters with good PIFs
            for j in range(len(winter_rasters)):
                if i != j:
                    rast2 = winter_rasters[j]
                    rast2 += str(band_num)
                    
                    pif_name = "pif.%s.%s" % (pif_count, rast1)
                    params = find_pif(grs.grass, rast1, rast2, pif_name, regression_eps=REGR_EPS, max_iteration=MAXITER)
                    
                    if params['good_quality']:
                        pif_count += 1
                    
                    pbar.set_postfix({'PIF rasters': pif_count})
                    pbar.update(1)
        pbar.close()

In [ ]:
winter_rasters = [
    'clean.topo.toar_LC81120282014042LGN00_B', 
    'clean.topo.toar_LC81120282014346LGN00_B', 
    'clean.topo.toar_LC81120282014362LGN00_B', 
    'clean.topo.toar_LC81120282015045LGN00_B', 
    'clean.topo.toar_LC81120282015077LGN00_B', 
    'clean.topo.toar_LC81120282015365LGN00_B', 
    'clean.topo.toar_LC81120282016016LGN00_B', 
    'clean.topo.toar_LC81120282016032LGN01_B', 
    'clean.topo.toar_LC81120282016048LGN00_B', 
    'clean.topo.toar_LC81120282016064LGN00_B', 
    'clean.topo.toar_LC81120282016080LGN00_B'
]
                
rast1 = 'clean.topo.toar_LC81120282014042LGN00_B1'
grs.grass.run_command('g.region', rast=rast1, res='120')

grs.grass.run_command('r.mask', flags='r')
grs.grass.run_command('r.mapcalc', expression="MASK=forest.mask.30@treecover", overwrite=True)

REGR_EPS = 0.1
MAXITER = 25

for band_num in range(1, 8):
    
    for i in range(len(winter_rasters)):
        
        iter_count = len(winter_rasters) - 1 
        rast1 = winter_rasters[i]
        rast1 += str(band_num)
        grs.grass.run_command('g.region', rast=rast1, res=120)
        
        with tqdm_notebook(total=iter_count, desc=rast1) as pbar:
            pif_count = 0  # count of rasters with good PIFs
            for j in range(len(winter_rasters)):
                if i != j:
                    rast2 = winter_rasters[j]
                    rast2 += str(band_num)
                    
                    pif_name = "pif.%s.%s" % (pif_count, rast1)
                    params = find_pif(grs.grass, rast1, rast2, pif_name, regression_eps=REGR_EPS, max_iteration=MAXITER)
                    
                    if params['good_quality']:
                        pif_count += 1
                    
                    pbar.set_postfix({'PIF rasters': pif_count})
                    pbar.update(1)
        pbar.close()

In [ ]:
winter_rasters = [
    'clean.topo.toar_LC81130272014049LGN00_B', 
    'clean.topo.toar_LC81130272014065LGN00_B', 
    'clean.topo.toar_LC81130272014337LGN00_B', 
    'clean.topo.toar_LC81130272014353LGN00_B', 
    'clean.topo.toar_LC81130272015036LGN00_B', 
    'clean.topo.toar_LC81130272015052LGN00_B', 
    'clean.topo.toar_LC81130272015068LGN00_B', 
    'clean.topo.toar_LC81130272015084LGN00_B', 
    'clean.topo.toar_LC81130272015340LGN00_B', 
    'clean.topo.toar_LC81130272015356LGN00_B', 
    'clean.topo.toar_LC81130272016039LGN00_B', 
    'clean.topo.toar_LC81130272016055LGN00_B', 
    'clean.topo.toar_LC81130272016071LGN00_B'
]
                
rast1 = 'clean.topo.toar_LC81130272014049LGN00_B1'
grs.grass.run_command('g.region', rast=rast1, res='120')

grs.grass.run_command('r.mask', flags='r')
grs.grass.run_command('r.mapcalc', expression="MASK=forest.mask.30@treecover", overwrite=True)

REGR_EPS = 0.1
MAXITER = 25

for band_num in range(1, 8):
    
    for i in range(len(winter_rasters)):
        
        iter_count = len(winter_rasters) - 1 
        rast1 = winter_rasters[i]
        rast1 += str(band_num)
        grs.grass.run_command('g.region', rast=rast1, res=120)
        
        with tqdm_notebook(total=iter_count, desc=rast1) as pbar:
            pif_count = 0  # count of rasters with good PIFs
            for j in range(len(winter_rasters)):
                if i != j:
                    rast2 = winter_rasters[j]
                    rast2 += str(band_num)
                    
                    pif_name = "pif.%s.%s" % (pif_count, rast1)
                    params = find_pif(grs.grass, rast1, rast2, pif_name, regression_eps=REGR_EPS, max_iteration=MAXITER)
                    
                    if params['good_quality']:
                        pif_count += 1
                    
                    pbar.set_postfix({'PIF rasters': pif_count})
                    pbar.update(1)
        pbar.close()


#### Аггрегация PIF

Посмотрим, насколько вообще совпадают PIF для одной сцены, полученные при расчетах относительно разных сцен. Для этого для каждой сцены подсчитаем попиксельную сумму PIF, а затем посмотрим распределение этой суммы.

Итак, видно, что если взять те пиксели, которые попали хотя бы в один растр PIF'ов, то по грубоей прикидке в среднем их будет около 50-70 тыс на сцену. Если учитывать те пиксели, которые были выбраны в качестве PIF дважды, то таких окажется около 20-40 тыс. на сцену. Если брать пиксели, выбранные трижды, то таких будет около 2-5 тыс. Остановимся в анализе на двух: создадим аггрегированный растр PIFов, который будет содержать в пикселе 0, если пиксель был выбран PIFом менее двух раз, иначе запишем в растр единицу:

In [ ]:
winter_rasters = [
    'clean.topo.toar_LC81120272014074LGN00_B', 
    'clean.topo.toar_LC81120272014346LGN00_B', 
    'clean.topo.toar_LC81120272014362LGN00_B', 
    'clean.topo.toar_LC81120272015045LGN00_B', 
    'clean.topo.toar_LC81120272015077LGN00_B', 
    'clean.topo.toar_LC81120272015333LGN00_B', 
    'clean.topo.toar_LC81120272015349LGN00_B',
    'clean.topo.toar_LC81120272015365LGN00_B',
    'clean.topo.toar_LC81120272016032LGN01_B', 
    'clean.topo.toar_LC81120272016048LGN00_B', 
    'clean.topo.toar_LC81120272016064LGN00_B', 
    'clean.topo.toar_LC81120272016080LGN00_B',
    
    'clean.topo.toar_LC81120282014042LGN00_B', 
    'clean.topo.toar_LC81120282014346LGN00_B', 
    'clean.topo.toar_LC81120282014362LGN00_B', 
    'clean.topo.toar_LC81120282015045LGN00_B', 
    'clean.topo.toar_LC81120282015077LGN00_B', 
    'clean.topo.toar_LC81120282015365LGN00_B', 
    'clean.topo.toar_LC81120282016016LGN00_B', 
    'clean.topo.toar_LC81120282016032LGN01_B', 
    'clean.topo.toar_LC81120282016048LGN00_B', 
    'clean.topo.toar_LC81120282016064LGN00_B', 
    'clean.topo.toar_LC81120282016080LGN00_B',
    
    'clean.topo.toar_LC81130272014049LGN00_B', 
    'clean.topo.toar_LC81130272014065LGN00_B', 
    'clean.topo.toar_LC81130272014337LGN00_B', 
    'clean.topo.toar_LC81130272014353LGN00_B', 
    'clean.topo.toar_LC81130272015036LGN00_B', 
    'clean.topo.toar_LC81130272015052LGN00_B', 
    'clean.topo.toar_LC81130272015068LGN00_B', 
    'clean.topo.toar_LC81130272015084LGN00_B', 
    'clean.topo.toar_LC81130272015340LGN00_B', 
    'clean.topo.toar_LC81130272015356LGN00_B', 
    'clean.topo.toar_LC81130272016039LGN00_B', 
    'clean.topo.toar_LC81130272016055LGN00_B', 
    'clean.topo.toar_LC81130272016071LGN00_B',
]

with tqdm_notebook(total=len(winter_rasters)) as pbar:
    
    for raster_pref in winter_rasters:
        for band_num in range(1, 8):
            raster = raster_pref + str(band_num)
            grs.grass.run_command('g.region', rast=raster, res=120)
            pifs = grs.grass.list_strings(type='rast', pattern='pif*' + raster)
            tmp = 'r' + uuid.uuid4().hex
            try:
                grs.grass.run_command('r.series', input=','.join(pifs), output=tmp, method='count')
                name = 'agg.pif.' + raster
                expr = "%s = if(%s > 1)" % (name, tmp)
                grs.grass.run_command('r.mapcalc', expression=expr, overwrite=True)
            finally:
                grs.grass.run_command('g.remove', type='rast', name=tmp, flags='f')
        pbar.update(1)
    pbar.close()

####  Рассчет нормированных растров

In [ ]:
with tqdm_notebook(total=len(winter_rasters)) as pbar:
    
    for raster_pref in winter_rasters:
        for band_num in range(1, 8):
            raster = raster_pref + str(band_num)
            grs.grass.run_command('g.region', rast=raster)
            
            grs.grass.run_command('r.mask', flags='r')
            grs.grass.run_command('r.mask', raster='forest.mask.30@treecover', overwrite=True)
            
            pif_name = 'agg.pif.' + raster
            coefs = get_norm_coef(grs.grass, rast1, pif_name)
            
            factor = coefs['factor']
            b = coefs['intercept']
            
            norm_name = 'norm.' + raster
            expr = "%s = %s*%s + %s" % (norm_name, factor, raster, b)
            grs.grass.run_command('r.mapcalc', expression=expr)
            
        pbar.update(1)
    pbar.close()


<a id='appendix1'></a>
### Приложение 1: лог поиска PIF

Поиск нормирующих коэффициентов внутри маски лесов

<a id='appendix2'></a>
### Приложение 2

Параметры нормализации, рассчитанные внутри маски лесов